In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "data/PlantVillage",
    shuffle=True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [4]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

## Train Validation Test Split

In [5]:
# Train Validation Test Split
# 80% Train, 10% Validation, 10% Test
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [6]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

## Cache, Shuffle, and Prefetch the Dataset

In [7]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

## Creating a Layer for Resizing and Normalization
normalize each image for each pixel from the range of 0 to 1.

In [8]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1.0/255),
])

Data Augmentation increases the accuracy of the model making it more robust to flipped, rotated, or zoomed images

In [9]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [13]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training = True), y)
).prefetch(buffer_size = tf.data.AUTOTUNE)

# Modelling
Convulotion Neural Network is used with layers for resizing, normalization, and data augmentation

In [14]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

In [15]:
model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation="relu", input_shape = input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax')
])

model.build(input_shape = input_shape)

In [16]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (32, 127, 127, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                  

## Model Compilation
Optimizer: adam <br>
Losses: SparseCategorialCrossentropy <br>
Metric: Accuracy

In [17]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
)

In [19]:
history = model.fit(
    train_ds,
    epochs = 50,
    batch_size = BATCH_SIZE,
    verbose = 1,
    validation_data = val_ds
)

Epoch 1/50
54/54 [==============================] - 85s 1s/step - loss: 0.9304 - accuracy: 0.4525 - val_loss: 0.8589 - val_accuracy: 0.4479
Epoch 2/50
54/54 [==============================] - 52s 961ms/step - loss: 0.7573 - accuracy: 0.6203 - val_loss: 0.5792 - val_accuracy: 0.7448
Epoch 3/50
54/54 [==============================] - 47s 863ms/step - loss: 0.4951 - accuracy: 0.7911 - val_loss: 0.4038 - val_accuracy: 0.8385
Epoch 4/50
54/54 [==============================] - 46s 839ms/step - loss: 0.3486 - accuracy: 0.8539 - val_loss: 0.3130 - val_accuracy: 0.8802
Epoch 5/50
54/54 [==============================] - 47s 868ms/step - loss: 0.3009 - accuracy: 0.8703 - val_loss: 0.3274 - val_accuracy: 0.8906
Epoch 6/50
54/54 [==============================] - 45s 847ms/step - loss: 0.2438 - accuracy: 0.9038 - val_loss: 0.2556 - val_accuracy: 0.9010
Epoch 7/50
54/54 [==============================] - 44s 808ms/step - loss: 0.1934 - accuracy: 0.9219 - val_loss: 0.2370 - val_accuracy: 0.9583
Ep

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
scores

100% accuracy is achieved with low loss

Plotting Loss and Accuracy curves

In [ ]:
histroy.history.keys()

In [ ]:
acc = history.history["accuracy"]
val_acc = history_history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

In [ ]:
plt.figure(figsize = (8,8))

plt.subplot(1, 2, 1)
plt.plot(range(50), acc, label = "Training Accuracy")
plt.plot(range(50), val_acc, label = "Validation Accuracy")
plt.legend(loc = "lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(range(50), loss, label = "Training Loss")
plt.plot(range(50), val_loss, label = "Validation Loss")
plt.legend(loc = "upper right")
plt.title("Training and Validation Loss")

plt.show()